# Notes and excercises from *Structure and Interpretation of Computer Programs*

The book is [available online on MIT's site](https://mitpress.mit.edu/sites/default/files/sicp/full-text/book/book.html), there are also [recorded lectures on MIT's OCW](https://www.youtube.com/playlist?list=PLE18841CABEA24090).

## 1. Building Abstractions with Procedures

In [1]:
(define π 3.14)

π

3.14

In [2]:
?assert

(assert OPERATOR EXPRESSION ANSWER): assert that (OPERATOR EXPRESSION ANSWER) is #t

In [3]:
(define (square x) (* x x))

(assert = (square 2) 4)
(assert = (square 3) 9)

ok

In [4]:
(define (circle-area r)
  (* π (square r)))

(assert = (circle-area 1) π)
(assert = (circle-area 2) 12.56)

ok

In [5]:
(define ε 1e-4)

(define (approx x y)
  (< (abs (- x y)) ε))

(assert = (approx 1 1) #t)
(assert = (approx (+ 1 (/ ε 2)) 1) #t)
(assert = (approx (- 1 (/ ε 2)) 1) #t)
(assert = (approx 2 1) #f)
(assert = (approx (+ 1 (* ε 1.1)) 1) #f)

ok

In [6]:
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x)
                 x)))

(define (improve guess x)
  (average guess (/ x guess)))

(define (average x y)
  (/ (+ x y) 2))

(define (good-enough? guess x)
  (< (abs (- (square guess) x)) 0.001))

(define (sqrt x)
  (sqrt-iter 1 x))

(assert approx (sqrt 4) 2)
(assert approx (sqrt 9) 3)

ok

### Internal definitions and block structure

In [7]:
(define (good-enough? guess x)
    (< (abs (- (square guess) x)) 0.001))

(define (improve guess x)
    (average guess (/ x guess)))

(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))

(assert approx (sqrt-iter 1 4) 2)
(assert approx (sqrt-iter 1 9) 3)

ok

> We would like to localize the subprocedures, hiding them inside `sqrt` so that `sqrt` could coexist with other successive approximations, each having its own private good-enough? procedure. [...]  Such nesting of definitions, called *block structure*, is basically the right solution to the simplest name-packaging problem.  [...] Since `x` is bound in the definition of sqrt, the procedures `good-enough?`, `improve`, and `sqrt-iter`, which are defined internally to sqrt, are in the scope of `x`. Thus, it is not necessary to pass x explicitly to each of these procedures. Instead, we allow `x` to be a free variable in the internal definitions, as shown below. Then `x` gets its value from the argument with which the enclosing procedure sqrt is called. This discipline is called *lexical scoping*.

In [8]:
(define (sqrt x)
  (define (good-enough? guess)
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess)
    (average guess (/ x guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess)
        guess
        (sqrt-iter (improve guess))))
  (sqrt-iter 1.0))

(assert approx (sqrt 4) 2)
(assert approx (sqrt 9) 3)

ok

In [9]:
;; local scope

(define (foo)
  (define π 3.1415)
  (list π))

(list (foo) π)

((3.1415) 3.14)

In [10]:
;; we can replace the methods easily

(define (bar x) (* 2 x))

(define (foo x) (bar x))

(foo 5)

10

In [11]:
(define (bar x) (* 100 x))

(foo 5)

500

**Exercise 1.6.**  Alyssa P. Hacker doesn't see why if needs to be provided as a special form. ``Why can't I just define it as an ordinary procedure in terms of `cond`?'' she asks. Alyssa's friend Eva Lu Ator claims this can indeed be done, and she defines a new version of `if`:

In [12]:
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

(assert = (new-if (= 2 3) 0 5) 5)
(assert = (new-if (= 1 1) 0 5) 0)

ok

In [13]:
(define (sqrt-iter-bad guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter-bad (improve guess x)
                         x)))

;; (sqrt-iter-2 1 2)

**Exercise 1.7.**  The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how guess changes from one iteration to the next and to stop when the change is a very small fraction of the `guess`. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers? 

In [14]:
(define (good-enough? guess x)
    (< (/ (abs (- (square guess) x)) guess) 0.001))

(assert approx (sqrt-iter 1.0 4) 2)
(assert approx (sqrt-iter 1.0 9) 3)

ok

**Exercise 1.8.**  Newton's method for cube roots is based on the fact that if $y$ is an approximation to the cube root of $x$, then a better approximation is given by the value 

$$
\frac{x \,/\, y^2 + 2y}{3}
$$

Use this formula to implement a cube-root procedure analogous to the square-root procedure. (In section 1.3.4 we will see how to implement Newton's method in general as an abstraction of these square-root and cube-root procedures.) 

In [15]:
(define (cube x) (* x x x))

(define (good-enough? guess x)
    (< (/ (abs (- (cube guess) x)) guess) 0.0001))

(define (improve guess x)
    (/ (+ (/ x (square guess)) (* 2 guess)) 3))

(define (cube-root x)
  (define (iter guess x)
    (if (good-enough? guess x)
        guess
        (iter (improve guess x) x)))
  (iter 1.0 x))

(assert approx (cube-root 27) 3)
(assert approx (cube-root (cube 18)) 18)

ok